In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [197]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [5]:
!pip install pycaret

     |████████████████████████████████| 301 kB 17.2 MB/s            
     |████████████████████████████████| 24.2 MB 21.7 MB/s            
     |████████████████████████████████| 15.6 MB 33.6 MB/s            
     |████████████████████████████████| 114 kB 31.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 10.4 MB 14.8 MB/s            
     |████████████████████████████████| 167 kB 34.6 MB/s            
     |████████████████████████████████| 636 kB 40.0 MB/s            
     |████████████████████████████████| 25.9 MB 24.9 MB/s            


     |████████████████████████████████| 1.0 MB 22.4 MB/s            
     |████████████████████████████████| 184 kB 28.3 MB/s            
     |████████████████████████████████| 14.8 MB 39.4 MB/s            
     |████████████████████████████████| 79 kB 14.5 MB/s            
     |████████████████████████████████| 58 kB 9.7 MB/s             
  Preparing metadata (setup.py) ... done


  Created wheel for pyod: filename=pyod-0.9.7-py3-none-any.whl size=136277 sha256=18b540475ac719eeac55ea8135972dc0a22a4dc3a14000008c1017874decd352
  Stored in directory: /root/.cache/pip/wheels/ce/14/ae/60cbb36511e59bc12f8f0883805f586db3b315972b54865d33
  Created wheel for databricks-cli: filename=databricks_cli-0.16.4-py3-none-any.whl size=106877 sha256=1148d99aa88e2610639a2a5f4e83debcfe20b8d0949c5c81c4ae6faf6ef782d6
  Stored in directory: /root/.cache/pip/wheels/a2/a1/6d/fa1d22ea25ed8593887437fe1c7e00f6ef307fc240ccd4dc5c
Successfully built pyod databricks-cli
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.2
    Uninstalling srsly-2.4.2:
      Successfully uninstall

In [6]:
from pycaret.regression import *

In [2]:
#import modules
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder,FunctionTransformer
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import r2_score
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
%matplotlib inline

In [7]:
KAGGLE=False

In [8]:
if KAGGLE:
    path='../input/jobathonfebruary2022/'
else:
    path=''

In [101]:
train_orig = pd.read_csv(path+'train_0OECtn8.csv')
test_orig = pd.read_csv(path+'test_1zqHu22.csv')

create inner test

In [136]:
kf = KFold(n_splits=8)
test_fold_gen = kf.split(train_orig,train_orig['engagement_score'])
sel_fold = 0
for fold,(train_index,test_index) in enumerate(test_fold_gen):
    if fold==sel_fold:
        test = train_orig.iloc[test_index,:].reset_index()
        train = train_orig.iloc[train_index,:].reset_index()
        break
print(train.shape)        
print(test.shape)

(78047, 11)
(11150, 11)


In [137]:
x= train['user_id'].value_counts()
user_id_cutoff = x[x>1].index

mask = train['user_id'].isin(user_id_cutoff)
train=train[mask].reset_index()
print(train.shape)

(75866, 12)


In [138]:
target_diff = set(test['engagement_score'].values).difference(set(train['engagement_score'].values))
print(len(target_diff),target_diff)
user_diff = set(test['user_id'].values).difference(set(train['user_id'].values))
print(len(user_diff),user_diff)
category_id_diff = set(test['category_id'].values).difference(set(train['category_id'].values))
print(len(category_id_diff),category_id_diff)
video_id_diff = set(test['video_id'].values).difference(set(train['video_id'].values))
print(len(video_id_diff),video_id_diff)
followers_diff = set(test['followers'].values).difference(set(train['followers'].values))
print(len(followers_diff),followers_diff)
views_diff = set(test['views'].values).difference(set(train['views'].values))
print(len(views_diff),views_diff)

0 set()
2334 {16385, 24578, 24579, 16390, 7, 8200, 8201, 24582, 16397, 16399, 19, 16405, 16425, 8235, 16430, 8239, 48, 8240, 8244, 16439, 24637, 16447, 24642, 8261, 16453, 8263, 72, 16454, 16456, 24650, 77, 24662, 16473, 99, 24675, 24684, 109, 16495, 112, 24694, 16504, 16508, 24707, 8326, 149, 16533, 8344, 8345, 16536, 8349, 24735, 16545, 24744, 24746, 172, 24751, 16563, 24755, 24762, 187, 8381, 24780, 24785, 210, 24787, 8407, 218, 16604, 221, 16605, 24796, 224, 24797, 24804, 8421, 16614, 234, 16618, 16629, 24828, 256, 24832, 24834, 8465, 274, 16662, 24856, 24857, 16667, 24861, 16677, 16678, 297, 8492, 306, 308, 8500, 16697, 24892, 319, 8511, 24895, 16706, 323, 325, 16711, 16712, 8522, 8523, 332, 24908, 336, 8530, 24915, 24919, 16728, 8541, 16734, 8543, 8548, 16742, 360, 24941, 16752, 24946, 373, 381, 8576, 386, 388, 8585, 16777, 8588, 24972, 8593, 24981, 407, 24983, 24985, 8602, 413, 24990, 16801, 24993, 8613, 8614, 8616, 8618, 428, 16813, 8622, 8627, 8631, 446, 25024, 8644, 25028, 16

In [139]:
mask1=test['user_id'].isin(list(user_diff))
test = test[~mask1].reset_index()
print(test.shape)

x= test['user_id'].value_counts()
print(x)
print(x[x==1].shape)

(8121, 12)
27150    5
11096    4
13162    4
11094    4
5614     4
        ..
27679    1
17046    1
20984    1
24358    1
2513     1
Name: user_id, Length: 7198, dtype: int64
(6369,)


In [140]:
print(train['user_id'].nunique(),test['user_id'].nunique())
print(train['category_id'].nunique(),test['category_id'].nunique())
print(train['video_id'].nunique(),test['video_id'].nunique())
print(train['followers'].nunique(),test['followers'].nunique())
print(train['views'].nunique(),test['views'].nunique())
print(train['engagement_score'].nunique(),test['engagement_score'].nunique())

25400 7198
47 46
175 173
17 17
43 42
229 229


In [141]:
test_target = test['engagement_score'].copy()
del test['engagement_score']

In [142]:
user_diff = set(test['user_id'].values).difference(set(train['user_id'].values))
print(len(user_diff),user_diff)

0 set()


In [143]:
print(train.shape)
print(test.shape)

(75866, 12)
(8121, 11)


In [148]:
def ordinal_dummy_coding(data,cols):
    data_updated = data.copy()
    for col in cols:
        enc_unique = np.sort(data[col].unique())
        start_enc = enc_unique[0]
        data_updated = pd.get_dummies(data_updated,columns = [col], prefix=[col])
        print('after_dummy_update:',data_updated.columns)
        for enc_val in enc_unique:
            #remove 1st dummy variable since it will become constant value
            #as all higher values of 1 will always update it to 1

            dummy_col = col+'_'+str(enc_val)
            if enc_val==start_enc:
                del data_updated[dummy_col]
                continue
                
            mask = data_updated[dummy_col]==1
            for prior_enc in range(start_enc+1,enc_val):
                prior_dummy_col = col+'_'+str(prior_enc)
                data_updated[prior_dummy_col][mask] = 1
            
    return data_updated    

In [149]:
print(np.sort(train['views'].unique()))
print(np.sort(test['views'].unique()))

[  30   44   52   72   89   95  134  138  156  178  180  229  248  309
  317  362  369  396  414  444  462  467  568  613  617  621  628  636
  662  689  709  714  781  806  819  840  862  884  892  900  909  990
 1000]
[  30   44   52   72   89   95  134  138  156  178  180  229  248  309
  317  362  369  396  414  444  462  467  568  613  617  621  628  636
  662  689  709  714  781  806  819  840  862  884  900  909  990 1000]


In [150]:
# keys = np.sort(train['followers'].unique())
# values = range(train['followers'].nunique())
# followers_map = dict(zip(keys, values))
# print(followers_map) 

In [151]:
# train['istrain']=1
# test['istrain']=0

# combined = pd.concat([train,test],axis=0)

In [152]:
# combined['followers'] = combined['followers'].replace(followers_map)
# combined= ordinal_dummy_coding(combined,['followers'])

In [153]:
# train= combined[combined['istrain']==1]
# test= combined[combined['istrain']==0]

# del combined,train['istrain'],test['istrain']

In [154]:
print(train.columns)
print(test.columns)

Index(['level_0', 'index', 'row_id', 'user_id', 'category_id', 'video_id',
       'age', 'gender', 'profession', 'followers', 'views',
       'engagement_score'],
      dtype='object')
Index(['level_0', 'index', 'row_id', 'user_id', 'category_id', 'video_id',
       'age', 'gender', 'profession', 'followers', 'views'],
      dtype='object')


In [155]:
def preproc(train,test):
    label_encoder = LabelEncoder()

    train['gender']= label_encoder.fit_transform(train['gender'])
    test['gender']= label_encoder.transform(test['gender'])
    
    #ordinal encoding of profession
    prof_map = {'Other':0,'Student':1,'Working Professional':2}
    train['profession'] = train['profession'].map(prof_map)
    test['profession'] = test['profession'].map(prof_map)
    
    #to have video id without target encoding
    train['video_id_raw']=train['video_id'].copy()
    
def gen_agg(train,test):
    #user_id aggregate features
    user_agg = train[['user_id','video_id','category_id']].groupby('user_id').agg('nunique').reset_index()
    user_agg = user_agg.rename({'video_id':'user_video_count','category_id':'user_category_count'},axis=1)
    train = pd.merge(train, user_agg, how='left', on='user_id')
    test = pd.merge(test, user_agg, how='left', on='user_id')
    
    
    #category_id aggregate features
    cat_agg = train[['category_id','user_id','video_id','age','gender','profession']] \
                        .groupby('category_id') \
                        .agg({'user_id':'nunique','video_id':'nunique', \
                             'age':'mean','gender':'mean','profession':'mean'}) \
                        .reset_index()
    cat_agg = cat_agg.rename({'user_id':'category_user_count',
                              'video_id':'category_video_count',
                              'age':'category_age_mean',
                              'gender':'category_gender_mean',
                              'profession':'category_profession_mean',
                             }, axis=1)
    train = pd.merge(train, cat_agg, how='left', on='category_id')
    test = pd.merge(test, cat_agg, how='left', on='category_id')    
    
    #video_id aggregate features
    video_agg = train[['video_id','age','gender','profession']] \
                        .groupby('video_id') \
                        .agg('mean') \
                        .reset_index() 
    video_agg = video_agg.rename({'age':'video_age_mean',
                              'gender':'video_gender_mean',
                              'profession':'video_profession_mean',
                             },axis=1)
    train = pd.merge(train, video_agg, how='left', on='video_id')
    test = pd.merge(test, video_agg, how='left', on='video_id')
    
    return train,test
        

In [156]:
preproc(train,test)

In [157]:
train,test = gen_agg(train,test)

In [158]:
print(train.columns)
train.head()

Index(['level_0', 'index', 'row_id', 'user_id', 'category_id', 'video_id',
       'age', 'gender', 'profession', 'followers', 'views', 'engagement_score',
       'video_id_raw', 'user_video_count', 'user_category_count',
       'category_user_count', 'category_video_count', 'category_age_mean',
       'category_gender_mean', 'category_profession_mean', 'video_age_mean',
       'video_gender_mean', 'video_profession_mean'],
      dtype='object')


,level_0,index,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score,video_id_raw,user_video_count,user_category_count,category_user_count,category_video_count,category_age_mean,category_gender_mean,category_profession_mean,video_age_mean,video_gender_mean,video_profession_mean
0,0,11150,11151,7939,8,17,15,1,1,280,628,4.77,17,2,1,2784,10,24.577807,0.611993,0.895946,25.179487,0.688363,0.954635
1,1,11151,11152,18620,21,122,17,1,1,320,909,3.73,122,3,2,2246,8,24.810670,0.589156,0.893012,24.950884,0.601179,0.905697
2,2,11152,11153,23392,17,37,19,0,1,210,884,2.70,37,3,2,661,2,25.022417,0.400585,0.948343,25.207031,0.388672,0.943359
3,4,11154,11155,17507,3,4,19,0,1,270,621,3.80,4,4,2,1204,2,25.457286,0.601759,0.895729,25.373300,0.592089,0.907293
4,5,11155,11156,6545,3,5,14,1,1,270,621,4.89,5,3,2,1204,2,25.457286,0.601759,0.895729,25.544061,0.611750,0.883780


In [159]:
print(np.sort(train['age'].unique()))
print(np.sort(train['followers'].unique()))
print(np.sort(train['views'].unique()))
print(np.sort(train['user_video_count'].unique())) 
print(np.sort(train['user_category_count'].unique()))#no bin
print(np.sort(train['category_user_count'].unique()))
print(np.sort(train['category_video_count'].unique()))

[10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57
 58 59 60 61 62 63 64 66 67 68]
[160 180 190 200 210 220 230 240 250 260 270 280 290 320 330 340 360]
[  30   44   52   72   89   95  134  138  156  178  180  229  248  309
  317  362  369  396  414  444  462  467  568  613  617  621  628  636
  662  689  709  714  781  806  819  840  862  884  892  900  909  990
 1000]
[ 2  3  4  5  6  7  8  9 10]
[1 2 3]
[  41  108  114  145  200  269  316  336  347  400  438  459  469  471
  503  572  580  585  619  657  661  716  717  790  868  882  939  942
  968  983 1143 1204 1225 1254 1416 1429 1499 1572 1585 1594 1615 1634
 2126 2156 2246 2511 2784]
[ 1  2  3  4  5  6  7  8 10 31]


In [160]:
def create_bins(combined):
    combined['age_bin']=pd.cut(combined.age,
                               bins=[9,18,25,31,36,41,51,60,70],
                                     labels = [1,2,3,4,5,6,7,8])
    print('age_bin:',combined['age_bin'].unique())
    combined['followers_bin']=pd.cut(combined.followers,
                                     bins=[159,200,240,270,290,360],
                                     labels = [1,2,3,4,5]
                                    )
    print('followers_bin:',combined['followers_bin'].unique())
    
    combined['views_bin']=pd.cut(combined.views,
                                     bins=[29,72,138,248,467,662,840,1000],
                                     labels = [1,2,3,4,5,6,7]
                                )
    print('views_bin:',combined['views_bin'].unique())   
    
    combined['category_user_count_bin']=pd.qcut(combined.category_user_count,[0, 0.15, .25, .5, .7, 0.8, 0.9, 1.],
                                         labels = [1,2,3,4,5,6,7])
    print('category_user_count_bin:',combined['category_user_count_bin'].unique())

    combined['user_video_count_bin']=pd.cut(combined.user_video_count,
                                     bins=[1,2,5,7,9,10],
                                     labels = [1,2,3,4,5]
                                           )
    print('user_video_count_bin:',combined['user_video_count_bin'].unique())   

    combined['category_video_count_bin']=pd.cut(combined.category_video_count,
                                     bins=[0,1,2,4,6,8,10,31],
                                     labels = [1,2,3,4,5,6,7])
    print('category_video_count_bin:',combined['category_video_count_bin'].unique())   
  

In [161]:
train['istrain']=1
test['istrain']=0

combined = pd.concat([train,test],axis=0)

In [162]:
create_bins(combined)

age_bin: [1, 2, 3, 4, 5, 6, 7, 8]
Categories (8, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7 < 8]
followers_bin: [4, 5, 2, 3, 1]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]
views_bin: [5, 7, 6, 4, 1, 2, 3]
Categories (7, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7]
category_user_count_bin: [7, 5, 1, 3, 4, 2, 6]
Categories (7, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7]
user_video_count_bin: [1, 2, 3, 4, 5]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]
category_video_count_bin: [6, 5, 2, 4, 3, 1, 7]
Categories (7, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7]


In [163]:
combined= ordinal_dummy_coding(combined,['age_bin'])
combined= ordinal_dummy_coding(combined,['followers_bin'])
combined= ordinal_dummy_coding(combined,['views_bin'])
combined= ordinal_dummy_coding(combined,['category_user_count_bin'])
combined= ordinal_dummy_coding(combined,['user_video_count_bin'])
combined= ordinal_dummy_coding(combined,['category_video_count_bin'])

combined= ordinal_dummy_coding(combined,['profession'])#without bin
combined= ordinal_dummy_coding(combined,['user_category_count'])#without bin

after_dummy_update: Index(['level_0', 'index', 'row_id', 'user_id', 'category_id', 'video_id',
       'age', 'gender', 'profession', 'followers', 'views', 'engagement_score',
       'video_id_raw', 'user_video_count', 'user_category_count',
       'category_user_count', 'category_video_count', 'category_age_mean',
       'category_gender_mean', 'category_profession_mean', 'video_age_mean',
       'video_gender_mean', 'video_profession_mean', 'istrain',
       'followers_bin', 'views_bin', 'category_user_count_bin',
       'user_video_count_bin', 'category_video_count_bin', 'age_bin_1',
       'age_bin_2', 'age_bin_3', 'age_bin_4', 'age_bin_5', 'age_bin_6',
       'age_bin_7', 'age_bin_8'],
      dtype='object')
after_dummy_update: Index(['level_0', 'index', 'row_id', 'user_id', 'category_id', 'video_id',
       'age', 'gender', 'profession', 'followers', 'views', 'engagement_score',
       'video_id_raw', 'user_video_count', 'user_category_count',
       'category_user_count', 'categor

In [164]:
print(len(combined.columns))
print(combined.columns)

59
Index(['level_0', 'index', 'row_id', 'user_id', 'category_id', 'video_id',
       'age', 'gender', 'followers', 'views', 'engagement_score',
       'video_id_raw', 'user_video_count', 'category_user_count',
       'category_video_count', 'category_age_mean', 'category_gender_mean',
       'category_profession_mean', 'video_age_mean', 'video_gender_mean',
       'video_profession_mean', 'istrain', 'age_bin_2', 'age_bin_3',
       'age_bin_4', 'age_bin_5', 'age_bin_6', 'age_bin_7', 'age_bin_8',
       'followers_bin_2', 'followers_bin_3', 'followers_bin_4',
       'followers_bin_5', 'views_bin_2', 'views_bin_3', 'views_bin_4',
       'views_bin_5', 'views_bin_6', 'views_bin_7',
       'category_user_count_bin_2', 'category_user_count_bin_3',
       'category_user_count_bin_4', 'category_user_count_bin_5',
       'category_user_count_bin_6', 'category_user_count_bin_7',
       'user_video_count_bin_2', 'user_video_count_bin_3',
       'user_video_count_bin_4', 'user_video_count_bin_5',

In [165]:
train= combined[combined['istrain']==1]
test= combined[combined['istrain']==0]

del combined,train['istrain'],test['istrain']

In [166]:
#remaining non bin columns 
#'user_id','category_id', 'video_id',  'gender'

In [167]:
targetcol = 'engagement_score'
target = train[targetcol]

In [168]:
actual_list = list(np.sort(train[targetcol].unique()))
unique_count  = train[targetcol].nunique()
replace_list  = range(1,unique_count+1)

keys = np.sort(train[targetcol].unique())
values = range(train[targetcol].nunique())
targetcol_map = dict(zip(keys, values))
# print(targetcol_map) 

targetcolbin  = targetcol+'_bin'
train[targetcolbin] = train[targetcol].replace(targetcol_map)
print(train[targetcolbin].unique())
train[[targetcol,targetcolbin]].head(25)

[208. 102.  40. 109. 218.  77.  11. 154. 114.  48. 117.  97. 157. 106.
 120.  76.   0.  21. 129. 160.   9.  27. 180. 171.  67. 214. 115. 201.
 149.  19.  18. 136.  37. 147. 125.  53. 140. 217.  57.  24.  99.  33.
  59. 151. 124.  31.  93.  17. 131.  60. 101. 177.  87. 144. 219.  69.
 184.  25. 163.  44.  90.  91.  65.  63.  20.  96.  75. 132.  68.  30.
  35.  83.  70. 224.  13. 100.  10.  28.  73. 112. 116.  62.  34.  43.
 121.  47. 197.  23. 103.  42. 123.  82. 167.  58. 220. 165.  26.  85.
 127.  86. 195. 221. 156. 216.  92. 105.  29.  94. 194. 200. 126.  55.
 168. 148.  98.  32. 179. 139. 178. 228. 199.  88.  95.   7.  84. 146.
   6. 118. 152. 187. 172. 138.  81. 110. 181. 202.  50. 133. 205. 137.
  38.   5.  80.  22.  78.  66.  16.  12. 113. 122. 119. 191.  74. 190.
  79. 134. 107.  15. 226.  61.  56.  51. 128.  39. 145. 143.  52.  36.
  89. 142. 183. 198.  72. 161. 206.  54. 169.  46.  71. 158.  49.   8.
 210. 204.   2. 193. 162. 176. 155. 150. 175. 186. 222. 130. 203. 227.
 173. 

,engagement_score,engagement_score_bin
0,4.77,208.0
1,3.73,102.0
2,2.70,40.0
3,3.80,109.0
4,4.89,218.0
5,3.40,77.0
6,1.39,11.0
7,4.27,154.0
8,3.85,114.0
9,2.89,48.0


In [169]:
train['video_id_orig']=train['video_id'].copy()
train['category_id_orig']=train['category_id'].copy()
train['user_id_orig']=train['user_id'].copy()

In [170]:
bin_cols =['age_bin','followers_bin','views_bin','category_user_count_bin',
           'user_video_count_bin','category_video_count_bin',]
remaining_cols = ['user_id','video_id','gender','category_id',
                 'profession_1', 'profession_2', 'user_category_count_2',
       'user_category_count_3']
orig_cols = ['user_id_orig','video_id_orig','category_id_orig']
# features_for_knn=bin_cols + remaining_cols + [targetcol]

features_for_knn=[]
for col in train.columns:
    for bin_col in bin_cols:
        if bin_col in col:
            features_for_knn+=[col]
features_for_knn +=   remaining_cols + orig_cols + [targetcol]          
print(features_for_knn)

['age_bin_2', 'age_bin_3', 'age_bin_4', 'age_bin_5', 'age_bin_6', 'age_bin_7', 'age_bin_8', 'followers_bin_2', 'followers_bin_3', 'followers_bin_4', 'followers_bin_5', 'views_bin_2', 'views_bin_3', 'views_bin_4', 'views_bin_5', 'views_bin_6', 'views_bin_7', 'category_user_count_bin_2', 'category_user_count_bin_3', 'category_user_count_bin_4', 'category_user_count_bin_5', 'category_user_count_bin_6', 'category_user_count_bin_7', 'user_video_count_bin_2', 'user_video_count_bin_3', 'user_video_count_bin_4', 'user_video_count_bin_5', 'category_video_count_bin_2', 'category_video_count_bin_3', 'category_video_count_bin_4', 'category_video_count_bin_5', 'category_video_count_bin_6', 'category_video_count_bin_7', 'user_id', 'video_id', 'gender', 'category_id', 'profession_1', 'profession_2', 'user_category_count_2', 'user_category_count_3', 'user_id_orig', 'video_id_orig', 'category_id_orig', 'engagement_score']


In [171]:
exclude_cols =['row_id', 'category_age_mean', 'category_gender_mean',
       'category_profession_mean',  'video_gender_mean',
       'video_profession_mean','video_id_raw',
            'video_age_mean',
        'video_age_ratio', 'video_profession_diff', 
        'video_profession_ratio', 'video_gender_diff', 'video_gender_ratio']
features =[col for col in train.columns if col not in exclude_cols]
print(len(features))
print(features)
features_excl_target = [col for col in features if col!=targetcol]

54
['level_0', 'index', 'user_id', 'category_id', 'video_id', 'age', 'gender', 'followers', 'views', 'engagement_score', 'user_video_count', 'category_user_count', 'category_video_count', 'age_bin_2', 'age_bin_3', 'age_bin_4', 'age_bin_5', 'age_bin_6', 'age_bin_7', 'age_bin_8', 'followers_bin_2', 'followers_bin_3', 'followers_bin_4', 'followers_bin_5', 'views_bin_2', 'views_bin_3', 'views_bin_4', 'views_bin_5', 'views_bin_6', 'views_bin_7', 'category_user_count_bin_2', 'category_user_count_bin_3', 'category_user_count_bin_4', 'category_user_count_bin_5', 'category_user_count_bin_6', 'category_user_count_bin_7', 'user_video_count_bin_2', 'user_video_count_bin_3', 'user_video_count_bin_4', 'user_video_count_bin_5', 'category_video_count_bin_2', 'category_video_count_bin_3', 'category_video_count_bin_4', 'category_video_count_bin_5', 'category_video_count_bin_6', 'category_video_count_bin_7', 'profession_1', 'profession_2', 'user_category_count_2', 'user_category_count_3', 'engagement_sco

In [172]:
catcols = []
numeric_cols = [col for col in features if col not in catcols+[targetcol]]
print(len(catcols))
print(catcols)
print(len(numeric_cols))
print(numeric_cols)

0
[]
53
['level_0', 'index', 'user_id', 'category_id', 'video_id', 'age', 'gender', 'followers', 'views', 'user_video_count', 'category_user_count', 'category_video_count', 'age_bin_2', 'age_bin_3', 'age_bin_4', 'age_bin_5', 'age_bin_6', 'age_bin_7', 'age_bin_8', 'followers_bin_2', 'followers_bin_3', 'followers_bin_4', 'followers_bin_5', 'views_bin_2', 'views_bin_3', 'views_bin_4', 'views_bin_5', 'views_bin_6', 'views_bin_7', 'category_user_count_bin_2', 'category_user_count_bin_3', 'category_user_count_bin_4', 'category_user_count_bin_5', 'category_user_count_bin_6', 'category_user_count_bin_7', 'user_video_count_bin_2', 'user_video_count_bin_3', 'user_video_count_bin_4', 'user_video_count_bin_5', 'category_video_count_bin_2', 'category_video_count_bin_3', 'category_video_count_bin_4', 'category_video_count_bin_5', 'category_video_count_bin_6', 'category_video_count_bin_7', 'profession_1', 'profession_2', 'user_category_count_2', 'user_category_count_3', 'engagement_score_bin', 'video

In [173]:
train.to_csv('train_preproc_knn.csv',index=False)
test.to_csv('test_preproc_knn.csv',index=False)

In [174]:
np.save('features_for_knn.npy', np.array(features_for_knn))

In [175]:
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.woe import WOEEncoder

In [176]:

ct = ColumnTransformer(
     [
         ("targetenc",  TargetEncoder(cols=['user_id','category_id', 'video_id'],
                                                min_samples_leaf=2, smoothing=0.1) , ['user_id','category_id', 'video_id']),
#       ("helmertenc",  HelmertEncoder() , ['followers']),
#     ("targetenc2",  TargetEncoder(cols=['followers'],
#                                                     min_samples_leaf=2, smoothing=0.1) , ['followers']),
#       ("rank",  FunctionTransformer(lambda x: x*10000) , ['followers']),
      ],remainder='passthrough')
# user_id_copy = ('user',FunctionTransformer(lambda x: x['user_id'].copy()))
# ct2 = ColumnTransformer(
#      [("rank",  FunctionTransformer(lambda x: x.rank()) , ['followers'])
#       ],remainder='passthrough')
# FeatureUnion([user_id_copy,ct])
preprocessor = ('preproc',ct)

In [204]:
min_samples_leaf=1
count =2
smoothing=0.1
prop = 1 / (1 + np.exp(-(count - min_samples_leaf) / smoothing))
print(prop)

0.9999546021312976


In [177]:
train[features_for_knn].head()

,age_bin_2,age_bin_3,age_bin_4,age_bin_5,age_bin_6,age_bin_7,age_bin_8,followers_bin_2,followers_bin_3,followers_bin_4,followers_bin_5,views_bin_2,views_bin_3,views_bin_4,views_bin_5,views_bin_6,views_bin_7,category_user_count_bin_2,category_user_count_bin_3,category_user_count_bin_4,category_user_count_bin_5,category_user_count_bin_6,category_user_count_bin_7,user_video_count_bin_2,user_video_count_bin_3,user_video_count_bin_4,user_video_count_bin_5,category_video_count_bin_2,category_video_count_bin_3,category_video_count_bin_4,category_video_count_bin_5,category_video_count_bin_6,category_video_count_bin_7,user_id,video_id,gender,category_id,profession_1,profession_2,user_category_count_2,user_category_count_3,user_id_orig,video_id_orig,category_id_orig,engagement_score
0,0,0,0,0,0,0,0,1,1,1,0,1,1,1,1,0,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,7939,17,1,8,1,0,0,0,7939,17,8,4.77
1,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,0,0,1,1,1,1,0,0,18620,122,1,21,1,0,1,0,18620,122,21,3.73
2,1,0,0,0,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,23392,37,0,17,1,0,1,0,23392,37,17,2.70
3,1,0,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,17507,4,0,3,1,0,1,0,17507,4,3,3.80
4,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,6545,5,1,3,1,0,1,0,6545,5,3,4.89


In [205]:
s = setup(train[features_for_knn], session_id=100,
          silent=True,
#           numeric_features=numeric_cols,
#           categorical_features=catcols,
          target = targetcol,
          normalize=True,
          custom_pipeline=preprocessor,
          fold=10,fold_shuffle=False,
          use_gpu=True,
          remove_perfect_collinearity=False,
          )

,Description,Value
0,session_id,100
1,Target,engagement_score
2,Original Data,"(75866, 45)"
3,Missing Values,False
4,Numeric Features,43
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(53106, 45)"


In [206]:
import pickle
fold_gen = get_config('fold_generator')
indices=[]
indicesfile = open('knn_kfold_gen', 'ab')
pickle.dump(fold_gen, indicesfile)                     
indicesfile.close()

In [207]:
# for reading also binary mode is important
dbfile = open('knn_kfold_gen', 'rb')     
result = pickle.load(dbfile)
for (train_index,test_index) in result.split(train):
    print('train index:',train_index)
    print('test index:',test_index)
dbfile.close()

train index: [ 7587  7588  7589 ... 75863 75864 75865]
test index: [   0    1    2 ... 7584 7585 7586]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [ 7587  7588  7589 ... 15171 15172 15173]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [15174 15175 15176 ... 22758 22759 22760]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [22761 22762 22763 ... 30345 30346 30347]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [30348 30349 30350 ... 37932 37933 37934]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [37935 37936 37937 ... 45519 45520 45521]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [45522 45523 45524 ... 53105 53106 53107]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [53108 53109 53110 ... 60691 60692 60693]
train index: [    0     1     2 ... 75863 75864 75865]
test index: [60694 60695 60696 ... 68277 68278 68279]
train index: [    0     1

In [208]:
X= get_config('X')
print(len(X.columns))
print(list(X.columns))
X.head()

45
['age_bin_2', 'age_bin_3', 'age_bin_4', 'age_bin_5', 'age_bin_6', 'age_bin_7', 'age_bin_8', 'followers_bin_2', 'followers_bin_3', 'followers_bin_4', 'followers_bin_5', 'views_bin_2', 'views_bin_3', 'views_bin_4', 'views_bin_5', 'views_bin_6', 'views_bin_7', 'category_user_count_bin_2', 'category_user_count_bin_3', 'category_user_count_bin_4', 'category_user_count_bin_5', 'category_user_count_bin_6', 'category_user_count_bin_7', 'user_video_count_bin_2', 'user_video_count_bin_3', 'user_video_count_bin_4', 'user_video_count_bin_5', 'category_video_count_bin_2', 'category_video_count_bin_3', 'category_video_count_bin_4', 'category_video_count_bin_5', 'category_video_count_bin_6', 'category_video_count_bin_7', 'user_id', 'video_id', 'category_id', 'profession_1', 'profession_2', 'user_category_count_2', 'user_category_count_3', 'user_id_orig', 'video_id_orig', 'category_id_orig', 'gender_0', 'gender_1']


,age_bin_2,age_bin_3,age_bin_4,age_bin_5,age_bin_6,age_bin_7,age_bin_8,followers_bin_2,followers_bin_3,followers_bin_4,...,category_id,profession_1,profession_2,user_category_count_2,user_category_count_3,user_id_orig,video_id_orig,category_id_orig,gender_0,gender_1
0,0,0,0,0,0,0,0,1,1,1,...,-0.888583,1,0,0,0,-0.734321,-1.258672,-0.888583,0.0,1.0
1,0,0,0,0,0,0,0,1,1,1,...,0.222118,1,0,1,0,0.599980,0.913196,0.222118,0.0,1.0
2,1,0,0,0,0,0,0,1,0,0,...,-0.119636,1,0,1,0,1.196111,-0.844982,-0.119636,1.0,0.0
3,1,0,0,0,0,0,0,1,1,0,...,-1.315775,1,0,1,0,0.460940,-1.527569,-1.315775,1.0,0.0
4,0,0,0,0,0,0,0,1,1,0,...,-1.315775,1,0,1,0,-0.908463,-1.506885,-1.315775,0.0,1.0


In [209]:
temp=train.sort_values(by='user_id_orig')
temp[['user_id_orig','video_id_orig','engagement_score']].head(100)

,user_id_orig,video_id_orig,engagement_score
18997,1,1,2.88
35826,1,2,3.66
59047,1,3,2.88
51931,2,6,2.76
57069,2,4,3.65
...,...,...,...
22443,33,83,2.48
729,34,59,1.84
57188,34,9,3.09
75140,34,87,3.13


In [188]:
%%time
knn_hamm = create_model('knn',n_neighbors=25,
                       metric='euclidean',
                       weights='distance',
                  )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.5197,0.4896,0.6997,0.3677,0.1972,0.3368
1,0.5117,0.4730,0.6877,0.3682,0.1903,0.2187
2,0.5160,0.4819,0.6942,0.3767,0.1926,0.2397
3,0.5214,0.4901,0.7001,0.3619,0.1953,0.2365
4,0.5122,0.4680,0.6841,0.3653,0.1863,0.2492
5,0.4995,0.4498,0.6707,0.3720,0.1813,0.2267
6,0.5176,0.4852,0.6966,0.3566,0.1929,0.2266
7,0.5197,0.4834,0.6953,0.3445,0.1891,0.2599
8,0.5019,0.4520,0.6723,0.3655,0.1807,0.2727
9,0.5116,0.4602,0.6784,0.3613,0.1810,0.2282


CPU times: user 7.21 s, sys: 358 ms, total: 7.57 s
Wall time: 34.6 s


In [210]:
%%time
#  params = {
#      'bagging_fraction': 0.8,
#     'bagging_freq': 15,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#      'max_depth': 12,
#     'num_leaves': 30,
#     'reg_alpha': 0.3,  
#     'reg_lambda': 0.7,
#      'n_estimators': 200,
#  }
lgbm = create_model('lightgbm')
lgbm

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.5214,0.4911,0.7008,0.3657,0.1956,0.3298
1,0.5101,0.4770,0.6907,0.3628,0.1890,0.2051
2,0.5032,0.4625,0.6801,0.4017,0.1879,0.2284
3,0.5180,0.4861,0.6972,0.3671,0.1933,0.2308
4,0.5104,0.4639,0.6811,0.3709,0.1835,0.2327
5,0.4975,0.4540,0.6738,0.3661,0.1807,0.2240
6,0.5094,0.4772,0.6908,0.3672,0.1909,0.2205
7,0.5161,0.4827,0.6948,0.3455,0.1880,0.2537
8,0.5052,0.4624,0.6800,0.3510,0.1818,0.2670
9,0.5043,0.4534,0.6733,0.3708,0.1787,0.2236


CPU times: user 40.2 s, sys: 143 ms, total: 40.4 s
Wall time: 8.46 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=100, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)